Use Cyber-ZHG's various Keras tools for NLP: Attention, Multi-head, etc. 
Use external model manager to allow freezing weights of basic model.
Text output, not syllables.

In [1]:
#!pip install numpy==1.16.1
!pip install keras==2.2.3
!pip install keras-multi-head keras-pos-embd
!wget -nc https://raw.githubusercontent.com/LanceNorskog/deep_meter_2/master/haiku_5.txt
!cut -f2 < haiku_5.txt | sort | uniq > haiku_5_short.txt
!wc -l haiku_5*.txt
from google.colab import drive
drive.mount('/content/gdrive')


     |████████████████████████████████| 317kB 2.8MB/s 
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4
  Created wheel for keras-multi-head: filename=keras_multi_head-0.20.0-cp36-none-any.whl size=15360 sha256=fad54bf7fd41f2798434864177dd5e43102275da11d4b76b2b796332a7079ec0
  Stored in directory: /root/.cache/pip/wheels/95/49/02/4eda210bc4c37ff1d45311665bceb790881dbea92b27b025a5
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7553 sha256=aa9ebef79c2884f6101c37f5d63d0556d5a82ebb87b1a998f912ee3d07b3b31f
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-self-attention: filename=keras_self_attention-0.41.0-cp36-none-any.whl size=17289 sha256=5202c98a50688f621f49292a362b9493cdae7743a64e0e138e00253d12bf2db3
  Stored in directory: /root/.cache/pip/wheels/cc/dc/17/84258b27a04cd38ac91998abe148203720ca6961

In [2]:
!pip install -q -U git+https://github.com/LanceNorskog/deep_meter_2#egg=deepmeter

In [3]:

from __future__ import print_function
import math
import pickle
import json
import os
import glob
import time

import numpy as np
import tensorflow as tf
import keras.backend as K
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model, Sequential, load_model
from keras import layers 
from keras import metrics
from keras.preprocessing import text
from sklearn.model_selection import train_test_split

from cmu.syllables_cmu import syllables as word2sylls
from cmu.mappers import Decoder, trim_homynyms
from cmu.full import FullSearch
from cmu.topk import get_top_k, decodewords, short_sentences
from cmu.wordmap import Wordmap
from cmu.readhaiku2 import Reader

from keras_multi_head import MultiHeadAttention
from keras_pos_embd import PositionEmbedding

#from cmu.report import find_top_k_match, report
from keras_stuff.loss import sparse_categorical_crossentropy as scc
#from keras_stuff.loss import sparse_categorical_crossentropy_temporal as scct
import keras_stuff.metrics as my_metrics
from keras_stuff.modelmgr import ModelManager

print(word2sylls['therefore'])

# number of total samples to use
max_data = 200000
# number of words for hashing trick
hash_mole = 20000
# number of output words in haiku
max_features = 140000
# longest output sentence
num_sylls = 5
# longest input sentence
max_words = 10
# what you think
batch_size = 32
# do not output the same haiku twice
deduplicate_haiku=False
# emit output as input
duplicate_haiku=True
# use long as input
use_big_text=True

model_base_file="/content/gdrive/My Drive/Colab Notebooks/haiku_text_base_5.h5"
model_file="/content/gdrive/My Drive/Colab Notebooks/haiku_text_5.h5"
model_base_file=model_file
print(model_file)


Using TensorFlow backend.


['DH EH R', 'F AO R']
/content/gdrive/My Drive/Colab Notebooks/haiku_text_5.h5


In [4]:
!date
print(word2sylls['door'])
#word2sylls = trim_homynyms(word2sylls)
print(word2sylls['door'])
decoder = Decoder(word2sylls)
syll2idx = decoder.syll2idx
idx2syll = decoder.idx2syll

print(syll2idx['DH EH R'], idx2syll[1])
print('# features: ', len(idx2syll))

for i in range(decoder.wordoff):
    decoder.wordlist[i] = 'word{}'.format(i)
    decoder.wordlength[i] = 1
for i in range(decoder.sylloff):
    decoder.idx2syll[i] = 'syll{}'.format(i)

big_haiku_file = "haiku_5.txt"
wordmap = Wordmap(len(decoder.wordlist))
reader = Reader(word2sylls, decoder)
(big_text, big_haiku) = reader.readfile(big_haiku_file, max_words=max_words, 
    deduplicate_haiku=deduplicate_haiku, duplicate_haiku=duplicate_haiku, max_data=max_data)
if use_big_text:
    input_text = big_text
else:
    input_text = big_haiku
big_hash = reader.gethash(input_text, max_words=max_words, hash_mole=hash_mole)
big_indexes = reader.getindexes(big_haiku, max_words=5)
print('{} -> {} : {}k, {}'.format(big_text[0], big_haiku[0], big_hash[0], big_indexes[0]))

print('Full length clauses: ', len(big_text))
print('Wordmap total entries: ', len(decoder.wordlist))
assert max_features > len(decoder.wordlist)

Wed Aug  7 02:14:03 UTC 2019
['D AO R']
['D AO R']
['<START>', '<STOP>', '<PAD>', '!exclamation-point', '"close-quote', '"double-quote', '"end-of-quote', '"end-quote', '"in-quotes', '"quote']
2443 0
# features:  15098
Haiku word not in word2sylls: caveman's
Haiku word not in word2sylls: smitty's
a white sink and door -> a white sink and door : [  309. 16042.   392.  7161.  3485.     0.     0.     0.     0.     0.]k, [    70 125741 106762   3649  32458]
Full length clauses:  200001
Wordmap total entries:  129476


In [5]:
# Split multiple datasets across same index
(train_i, test_i, _, _) = train_test_split(np.arange(len(big_text)), np.arange(len(big_text)))

train_len=(len(train_i)//batch_size) * batch_size
test_len=(len(test_i)//batch_size) * batch_size
x_train = big_hash[train_i][:train_len]
y_train = big_indexes[train_i][:train_len]
y_train = np.reshape(y_train, (-1, y_train.shape[1], 1))
x_test = big_hash[test_i][-test_len:]
y_test = big_indexes[test_i][-test_len:]
y_test = np.reshape(y_test, (-1, y_test.shape[1], 1))

print(x_train.shape)
print(y_train.shape)

print(input_text[train_i[0]], x_train[0], str(x_test[0]))

def get_lstm(size, return_sequences=True):
    #return layers.LSTM(size, return_sequences=return_sequences)
    return layers.CuDNNLSTM(size, return_sequences=return_sequences)




(149984, 10)
(149984, 5, 1)
an extended bus with bicycle on front rack [ 2239.  7812. 18023. 15095.  7795.   815. 15399. 18606.     0.     0.] [14233.  7457.     0.     0.     0.     0.     0.     0.     0.     0.]


In [6]:
modelmgr = ModelManager()

embed_size=512
units_k=embed_size
units_v=embed_size
units_v=embed_size//3
units=512
dropout=0.5

model_params = {
    'max_words':max_words , 'hash_mole': hash_mole, 
    'embed_size': embed_size, 'units': units, 
    'num_sylls': num_sylls, 'max_features': max_features
    }

metric_list = [my_metrics.sparse, my_metrics.perfect]
metric_names = ['sparse', 'perfect']

model = modelmgr.get_model(model_params)

model.compile('adam', loss='sparse_categorical_crossentropy', metrics=metric_list)
model.summary()

# try using different optimizers and different optimizer configs

print('Train...')
history = None
use_saved_model=True
if not use_saved_model or not os.path.exists(model_base_file):
  with tf.Session() as session:
    K.manual_variable_initialization(False)
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    if os.path.exists(model_base_file):
        #print("Reloading weights")
        #model.load_weights(model_base_file, by_name=True)
        #modelmgr.unfreeze(model, 'dense_1')
        pass
    
    history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=300,
          callbacks=[EarlyStopping(monitor='val_perfect', mode='max', verbose=1, patience=10),
            ModelCheckpoint(model_file, monitor='val_perfect', save_best_only=True, save_weights_only=True, mode='max', verbose=1)],
          verbose=2,
          validation_data=[x_test, y_test])


W0807 02:14:13.818059 139848649914240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0807 02:14:13.864301 139848649914240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 02:14:13.870989 139848649914240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0807 02:14:13.885796 139848649914240 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0807 02:14:13.895082 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 512)           10240000  
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 512)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               1576960   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 512)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 5, 512)            0         
__________

In [7]:

plt.figure()
if history != None:
  # summarize history for accuracy
  for m in metric_names:
      #plt.plot(history.history[m])
      plt.plot(history.history['val_' + m])
  plt.title('model accuracy (dropout={})'.format(dropout))
  plt.xlabel('epoch')
  sname = []
  for m in metric_names:
      sname.append('{}={:01.3f}'.format(m, history.history['val_' + m][-1]))
  plt.legend(sname, loc='lower right')
  plt.show()

<Figure size 432x288 with 0 Axes>

In [8]:
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights(model_file)  
  print('x_test.shape ', x_test.shape)
  print('y_text.shape ', y_test.shape)
  eval_small = model.evaluate(x_test, y_test)
  print('model.evaluate on test data: ' ,model.metrics_names, eval_small)
  print('history: ', history)


x_test.shape  (49984, 10)
y_text.shape  (49984, 5, 1)
49984/49984 [==============================] - 93s 2ms/step
model.evaluate on test data:  ['loss', 'sparse', 'perfect'] [0.0968059579491205, 0.9858314633445764, 0.9645486555697823]
history:  None


In [0]:
def find_top_k_match(data, prediction, top_k=5):
        out = [-1] * len(data)
        for i in range(len(data)):
            topind = np.argsort(prediction[i])
            topind = topind[-top_k:]
            for j in range(top_k):
                #print(data[i][0], topind[j])
                if data[i][0] == topind[j]:
                    out[i] = topind[j]
        return out
    
def report(data, prediction):
    def match(data, prediction):
        assert len(data.shape) == 2
        assert len(prediction.shape) == 2
        good = 0
        top5 = 0
        count = 0
        for i in range(len(data)):
            topind = np.argsort(prediction[i])
            if data[i][0] == topind[-1]:
                good += 1
            topind = topind[-5:len(topind)]
            for j in range(5):
                if data[i][0] == topind[j]:
                    top5 += 1
                    break
            count += 1
        return (good, top5, count)

    _sparse = 0.0
    _perfect = 0.0
    _sparse5 = 0.0
    _perfect5 = 0.0
    _total = 0
    for n in range(len(data)):
        #print(len(short[n]))
        (good, top5, count) = match(data[n], predicts[n])
        if count == 0:
            continue
        _sparse += good/count
        _sparse5 += top5/count
        if good == count:
            _perfect += 1  
        if top5 == count:
            _perfect5 += 1
        _total += 1
    return {'sparse':_sparse/_total, 'perfect': _perfect/_total, 'sparse5': _sparse5/_total, 'perfect5': _perfect5/_total}


In [10]:
top_k=2

try_indx=x_test
try_text=input_text[test_i]
#try_indx=x_train
#try_text=input_text[train_i]
   
bigbatch = batch_size * 32
with tf.Session() as session:
  K.set_session(session)
  session.run(tf.global_variables_initializer())
  session.run(tf.tables_initializer())
  model.load_weights(model_file)  
  biglen = len(x_test)
  fs = FullSearch(num_sylls * 5, num_sylls, top_k)
  for i in range(0, biglen, bigbatch):
      predicts = model.predict(try_indx[i:i + bigbatch], batch_size=bigbatch)
      for j in range(0, len(predicts)):
          #f = find_top_k_match(y_test[i + j], predicts[j], 5)
          #if np.min(f) > 0 and j == 0:
          #    print('{} -> {}'.format(x_test[i + j], [decoder.idx2syll[k] for k in f]))
          (top_vals, top_paths) = get_top_k(predicts[j], top_k=top_k)
          fs.mainloop(top_paths)
          sentences = decodewords(fs.scorepaths, top_paths, decoder.wordlist)
          if len(sentences) > 0:
              for s in short_sentences(sentences, num_sylls):
                    print('{} -> {}'.format(try_text[i + j], s))
              #print('{} -> {}'.format(x_test[i + j], sentences[0]))
              #for k in range(1, len(sentences)):
              #      print('. -> {}'.format(sentences[k]))
    

Exception ignored in: <bound method BaseSession._Callable.__del__ of <tensorflow.python.client.session.BaseSession._Callable object at 0x7f30594fee10>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 1473, in __del__
    self._session._session, self._handle)
tensorflow.python.framework.errors_impl.CancelledError: (None, None, 'Session has been closed.')


(5, 140000)
predict.shape:  (5, 2)
through a clear blue sky -> roman numerals numerals roman roman
through a clear blue sky -> partially numerals numerals roman roman
through a clear blue sky -> roman instructions numerals roman roman
through a clear blue sky -> roman numerals word0 roman roman
through a clear blue sky -> partially numerals word0 roman roman
through a clear blue sky -> roman numerals numerals roman word0
through a clear blue sky -> roman numerals numerals word0 roman
through a clear blue sky -> roman instructions word0 roman roman
through a clear blue sky -> partially numerals numerals word0 roman
through a clear blue sky -> partially numerals numerals roman word0
through a clear blue sky -> roman instructions numerals roman word0
through a clear blue sky -> roman instructions numerals word0 roman
through a clear blue sky -> roman numerals word0 roman word0
through a clear blue sky -> roman numerals word0 word0 roman
through a clear blue sky -> partially numerals word0

KeyboardInterrupt: ignored